In [38]:
from pathlib import Path
import pandas as pd
from sqlite3 import connect

In [39]:
current_dir = Path.cwd()
if current_dir.name == "lab":
    current_dir = current_dir.parent
data = current_dir / "data"
db = data / "saved_videos.db"

In [40]:
db_conn = connect(str(db))

In [41]:
prompt_results = pd.read_sql("SELECT * FROM PromptResults", db_conn)

In [42]:
samples = []
SYSTEM_PROMPT = "You summarize multiple video transcripts into concise, factual summaries."

for row in prompt_results[["AgentPrompt", "Response"]].itertuples(index=False):
    samples.append({
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": row[0]},
            {"role": "assistant", "content": row[1]},
        ]
    })

In [43]:
samples[0]

{'messages': [{'role': 'system',
   'content': 'You summarize multiple video transcripts into concise, factual summaries.'},
  {'role': 'user',
   'content': "USER PROMPT: how to make canary pudding\nVIDEO TRANSCRIPTS :-\n\nTITLE_OF_VIDEO_RESULT_1 Canary pudding with jam sauce\n\nTRANSCRIPT_OF_VIDEO_RESULT_1: [Music] \n[Music] \nforeign \nbe \nis \n[Music] \n[Music] \n[Music] \nforeign \n[Music] \nforeign \nis \n\n\n\nTITLE_OF_VIDEO_RESULT_2 How to Make Homemade Vanilla Pudding | Pudding Recipes | Allrecipes.com\n\nTRANSCRIPT_OF_VIDEO_RESULT_2: [Music] \nall recipes member Rosemary submits this \ndelectable recipe for homemade vanilla \npudding you'll be tempted to eat this \nwarm so you might want to make a double \nbatch for \nlater to begin in a medium saucepan over \nmedium heat warm two cups of milk until \nbubbles form at the edges we're using \nwhole milk for an extra rich flavor \nnow in a bowl combine 1/2 cup of \nsugar 3 tbsp of corn \nstarch and 1/4 teaspoon of \nsalt slowly

In [44]:
indices = {13, 47, 56, 60, 77, 85, 103, 108, 109, 132, 134, 142, 156, 169, 173, 178, 211, 225, 232, 236, 244, 253, 279, 281, 284, 291, 293, 324, 338, 341, 342, 368, 374, 381, 394, 396, 397, 403, 415, 437, 451, 456, 459, 487, 542, 548, 553, 573, 582, 600, 605, 629, 630, 671, 677, 685, 694, 711, 736, 738, 741, 757, 763, 765, 771, 780, 784, 789, 803, 819, 827, 834, 843, 852, 860, 863, 876, 888, 897, 910, 911, 933, 940, 942, 948, 967, 986, 998, 1005, 1010, 1016, 1019, 1022, 1027, 1058, 21, 63, 65, 68, 171, 308, 310, 344, 354, 363, 370, 376, 412, 562, 563, 574, 590, 625, 680, 699, 708, 742, 743, 799, 961, 980, 1020, 62, 84, 98, 100, 144, 145, 213, 260, 335, 391, 452, 466, 485, 496, 567, 626, 675, 731, 755, 796, 838, 848, 849, 954, 1013, 1056, 128, 795, 828, 908, 1035}
print("Loss:", len(indices))

Loss: 153


In [45]:
max(indices) - 1, len(samples)

(1057, 1073)

In [46]:
samples = [value for index, value in enumerate(samples) if index not in indices]

In [48]:
len(samples), len(indices)

(920, 153)

In [49]:
pd.DataFrame(samples).to_json(data / "custom_dataset.jsonl", orient='records', lines=True)

In [50]:
prompt_results = pd.read_sql("SELECT * FROM PromptResults", db_conn)

In [58]:
flagged = prompt_results.iloc[list(indices)]

Key insights from VIDEO_1:  
VIDEO_1 shows a full DIY backsplash installation using subway tile, starting with removing the old backsplash, prepping the wall, pulling outlet covers, and protecting countertops. The creator explains how to cut tiles with both a wet saw and a grinder, showing straight, L shaped, and U shaped cuts. He demonstrates applying pre mixed mortar, placing spacers, alternating tile rows, and working around outlets carefully. After tiling, he grouts with unsanded grout, wipes haze repeatedly, reinstalls outlets with longer screws, and finishes the edges with caulk. The overall tone encourages beginners and stresses that this is a simple weekend project.

Key insights from VIDEO_2:  
VIDEO_2 provides a thorough step by step demonstration that not only covers installation but also emphasizes materials, costs, and layout planning. It explains how to calculate square footage, choose the right adhesive, pick spacers, and use tools like laser levels, wet saws, and diamon

In [62]:
sample = flagged.sample().iloc[0]
print(sample.Response)

For VIDEO_1, the speaker explains in a simple and direct way how pesticides kill ants by disrupting their muscles and nervous system. The description focuses on how insecticides cause paralysis that eventually leads to death, using the example of a white line people draw to stop ants from crossing. The content stays tightly centered on the mechanism of action of common pesticides and does not address broader environmental or health issues.

For VIDEO_2, the discussion is entirely unrelated to DDT or pesticides. It focuses on a Roblox game called Steal a Brain Rot, describing how players steal creatures from one another, how kids bypass age restrictions for voice chat, and how the game resembles older Fortnite style trading videos. The tone is playful and light, and the content stays centered on gaming behavior rather than any environmental or scientific topic.

For VIDEO_3, the speaker gives a short, direct answer about why DDT remains controversial. They list environmental impact, hea